In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")

portfolio = portfolio.set_index("Date")
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()

portfolio.loc[:,"LQDCumDiv"]=portfolio.loc[:,"LQD Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueLQD"]=portfolio.loc[:,"LQD Position"]+portfolio.loc[:,"LQDCumDiv"]
portfolio = portfolio.sort_index()

LQDCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueLQD"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
LQDCorrData.loc[:,"TotalReturnsLQD"] = LQDCorrData["TotalValueLQD"].pct_change()
LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})
LQDCorrData.loc[:,"HY_Index_OAS_Diff"]=LQDCorrData["HY_Index_OAS"].diff()
LQDCorrData.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=LQDCorrData["HY_INDEX_Z_SPREAD"].diff()

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
LQDCorrData.loc[:,"10yYieldDiff"] = LQDCorrData["10y_UST_INDEX_PX"].diff()

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})
LQDCorrData.loc[:,"IG_Index_OAS_Diff"]=LQDCorrData["IG_Index_OAS"].diff()
LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_SPREAD"})
LQDCorrData.loc[:,"IG_Index_Z_SPREAD_Diff"]=LQDCorrData["IG_INDEX_Z_SPREAD"].diff()

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(portfolio.loc["2021-01-11":"2025-12-31","SPY Position"]/-10,left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"SPY Position":"SPY_LAST_PX"})
LQDCorrData.loc[:,"ReturnSPY"] = LQDCorrData["SPY_LAST_PX"].pct_change()

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

LQDCorrData = LQDCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
LQDCorrData = LQDCorrData.rename(columns={"PX_LAST":"Vix Idx"})

LQDCorrData=LQDCorrData.dropna()
LQDCorrData = LQDCorrData.sort_index(ascending=False)

In [5]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]
LQDCorrData.loc[:,"PC3"]=X_pca[:,2]
LQDCorrData.loc[:,"PC4"]=X_pca[:,3]
LQDCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(LQDCorrData[["PC1","PC2","PC3","PC4"]])
Y = LQDCorrData["TotalReturnsLQD"]/200

model = sm.OLS(Y,X).fit()
model.summary()

[0.37592459 0.20547558 0.1983754  0.17148614 0.04873829]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Diff  ReturnSPY  MOVE Idx
0           0.668829     -0.344596           0.653144   0.001145  0.085570
1          -0.017233      0.179184           0.011969   0.628606  0.756510
2          -0.035617     -0.382317          -0.096966   0.749045 -0.531126
3           0.194933      0.834621           0.289049   0.209174 -0.371626
4           0.716312      0.079926          -0.693042  -0.005626  0.013025


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     1723.
Date:                Wed, 14 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:48:13   Log-Likelihood:                 12510.
No. Observations:                1240   AIC:                        -2.501e+04
Df Residuals:                    1235   BIC:                        -2.498e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9.605e-08   2.86e-07     -0.336      0.737   -6.57e-07    4.65e-07
PC1         2.378e-06   2.09e-07     11.399      0.000    1.97e-06    2.79e-06
PC2        -4.114e-06   2.82e-07    -14.577      0.000   -4.67e-06   -3.56e-06
PC3         9.664e-06   2.87e-07     33.645      0.000     9.1e-06    1.02e-05
PC4        -2.274e-05   3.09e-07    -73.615      0.000   -2.33e-05   -2.21e-05
==============================================================================
Omnibus:                       64.625   Durbin-Watson:                   2.558
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              211.141
Skew:                          -0.133   Prob(JB):                     1.42e-46
Kurtosis:                       5.004   Cond. No.                         1.48
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["10yYieldDiff","IG_Index_OAS_Diff","ReturnSPY","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]
LQDCorrData.loc[:,"PC3"]=X_pca[:,2]
LQDCorrData.loc[:,"PC4"]=X_pca[:,3]


X = sm.add_constant(LQDCorrData[["PC1","PC2","PC3","PC4"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.29961408 0.25665511 0.24728963 0.19644118]
   10yYieldDiff  IG_Index_OAS_Diff  ReturnSPY  MOVE Idx
0     -0.670939           0.712557   0.056859  0.197157
1      0.252836          -0.019333   0.602955  0.756403
2     -0.253414          -0.150487   0.785002 -0.544891
3      0.649382           0.685012   0.130326 -0.303441


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.855
Model:                            OLS   Adj. R-squared:                  0.855
Method:                 Least Squares   F-statistic:                     1825.
Date:                Wed, 14 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:52:11   Log-Likelihood:                 5970.2
No. Observations:                1240   AIC:                        -1.193e+04
Df Residuals:                    1235   BIC:                        -1.190e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05   5.58e-05     -0.344      0.731      -0.000    9.03e-05
PC1            0.0019    5.1e-05     37.927      0.000       0.002       0.002
PC2           -0.0011   5.51e-05    -20.275      0.000      -0.001      -0.001
PC3            0.0014   5.61e-05     25.144      0.000       0.001       0.002
PC4           -0.0044    6.3e-05    -69.404      0.000      -0.004      -0.004
==============================================================================
Omnibus:                       66.278   Durbin-Watson:                   2.586
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              236.787
Skew:                          -0.065   Prob(JB):                     3.82e-52
Kurtosis:                       5.137   Cond. No.                         1.23
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["HY_Index_OAS_Diff","10yYieldDiff","ReturnSPY","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","ReturnSPY","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]
LQDCorrData.loc[:,"PC3"]=X_pca[:,2]
LQDCorrData.loc[:,"PC4"]=X_pca[:,3]

X = sm.add_constant(LQDCorrData[["PC1","PC2","PC3","PC4"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.31807817 0.25680334 0.24520695 0.17991154]
   HY_Index_OAS_Diff  10yYieldDiff  ReturnSPY  MOVE Idx
0           0.708161     -0.697781   0.053710  0.093403
1          -0.007376      0.142415   0.651183  0.745402
2          -0.110323     -0.138683   0.753627 -0.632963
3           0.697339      0.688176   0.071573 -0.187107


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     1247.
Date:                Wed, 14 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:52:07   Log-Likelihood:                 5774.3
No. Observations:                1240   AIC:                        -1.154e+04
Df Residuals:                    1235   BIC:                        -1.151e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05   6.54e-05     -0.294      0.769      -0.000       0.000
PC1            0.0024    5.8e-05     41.222      0.000       0.002       0.003
PC2           -0.0007   6.45e-05    -10.093      0.000      -0.001      -0.001
PC3            0.0008    6.6e-05     11.883      0.000       0.001       0.001
PC4           -0.0043   7.71e-05    -55.172      0.000      -0.004      -0.004
==============================================================================
Omnibus:                       79.319   Durbin-Watson:                   2.397
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              339.288
Skew:                          -0.028   Prob(JB):                     2.11e-74
Kurtosis:                       5.562   Cond. No.                         1.33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["10yYieldDiff","ReturnSPY","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["10yYieldDiff","ReturnSPY","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]
LQDCorrData.loc[:,"PC3"]=X_pca[:,2]


X = sm.add_constant(LQDCorrData[["PC1","PC2","PC3"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.3424918  0.33903317 0.31847503]
   10yYieldDiff  ReturnSPY  MOVE Idx
0      0.003647   0.705805  0.708396
1      0.850135  -0.375201  0.369452
2     -0.526552  -0.600886  0.601398


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     1159.
Date:                Wed, 14 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:52:02   Log-Likelihood:                 5601.5
No. Observations:                1240   AIC:                        -1.120e+04
Df Residuals:                    1236   BIC:                        -1.117e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05   7.51e-05     -0.256      0.798      -0.000       0.000
PC1        -4.384e-05   7.41e-05     -0.591      0.554      -0.000       0.000
PC2           -0.0038   7.45e-05    -50.767      0.000      -0.004      -0.004
PC3            0.0023   7.69e-05     29.979      0.000       0.002       0.002
==============================================================================
Omnibus:                      117.774   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              659.781
Skew:                           0.219   Prob(JB):                    5.38e-144
Kurtosis:                       6.547   Cond. No.                         1.04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(LQDCorrData[["10yYieldDiff","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["10yYieldDiff","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

LQDCorrData.loc[:,"PC1"]=X_pca[:,0]
LQDCorrData.loc[:,"PC2"]=X_pca[:,1]

X = sm.add_constant(LQDCorrData[["PC1","PC2"]])
Y = LQDCorrData["TotalReturnsLQD"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.50977854 0.49022146]
   10yYieldDiff  MOVE Idx
0      0.707107  0.707107
1      0.707107 -0.707107


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsLQD   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     1740.
Date:                Wed, 14 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:52:57   Log-Likelihood:                 5601.5
No. Observations:                1240   AIC:                        -1.120e+04
Df Residuals:                    1237   BIC:                        -1.118e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.921e-05   7.51e-05     -0.256      0.798      -0.000       0.000
PC1           -0.0032   7.44e-05    -42.508      0.000      -0.003      -0.003
PC2           -0.0031   7.59e-05    -40.893      0.000      -0.003      -0.003
==============================================================================
Omnibus:                      117.680   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              658.596
Skew:                           0.219   Prob(JB):                    9.72e-144
Kurtosis:                       6.543   Cond. No.                         1.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""